<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_02_RNN_LSTM_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse

!pip install unidecode
import unidecode
import string
import random
import re
import time, math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.6 MB/s eta 0:00:00


In [2]:
num_epochs = 1000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [3]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
100


In [4]:
import tensorflow as tf
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',cache_subdir='/content/sample_data')  # shakespeare
# 학습에 사용할 txt 파일을 읽습니다.
file = open(data_dir, 'rb').read().decode(encoding='utf-8')
file_len = len(file)
print('file_len =', file_len)

1115394/1115394 [==============================] - 1s 1us/step
file_len = 1115394


In [5]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

o just is God, to right the innocent.

HASTINGS:
O, 'twas the foulest deed to slay that babe,
And the most merciless that e'er was heard of!

RIVERS:
Tyrants themselves wept when it was reported.

DORS


In [6]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [7]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [8]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)#.cuda()

In [10]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [12]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [13]:
def test(seq):
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(seq):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [14]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        target = label[j]
        y,hidden,cell = model(x,hidden,cell)
        y = y.squeeze()
        target = target.squeeze()
        loss += loss_func(y,target)
    loss.backward()
    optimizer.step()
  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
    if (i) % 50 == 0:
      print ('------------------------------------------------------')
      print ('Epoch {} Loss {:.4f}'.format(i+1, loss))
      print("샘플링을 시작합니다!")
      print ('------------------------------------------------------')
      test(200)
      print('\n')
      print ('------------------------------------------------------')
      print("샘플링이 끝났습니다.!")

------------------------------------------------------
Epoch 1 Loss 918.5844
샘플링을 시작합니다!
------------------------------------------------------
bC#xy4<(!5sm/C
*I
Rq}r&*a%"'nfDF,6__T"HJ]~ n^
o-SR~;8inVB[,e8IdXKH45Hu9k?~g'
C
ep}$>:,#^DT[VN"%MqG{r^>]pzu"{NysZ0
zH~h/6XVUmEn(8Ke;Hg8K:w`:CD@$l'pzh\ta8dS_L;R(B/%oSO`YNuy#~{^V69~nhf)aEfQp(giz5M??e2?]n4yr<+yVR_)+~~xUvVQ!Ga)0bwjxk)b)o
X	,8];^gVE)>^\lJ]&V9iO;<*G	v7'wp-P]oOPKZh&Tp>m$kgpLMixTbBB)a:<J 0-.90{ DVtYT+aE*>Vcj9%;[O\cA
3XYYS	0,,2i+$3CZ\__7=0atlT!W^xJspk\#O%]xdO'2$Nt)
U\$/G$<`
}s9MmbJ>k9i"2G)?@tW2	sj*3L4j&{.R&IE
8	+J<9n997B*W5'E4
oyp!
>s!:L?m\5?e
!E(:yKk[R~?BXj0?3MWbE/
rb_"1hg^rS6M9`tV_.tz]8V?	&{HVzx"&!
i'~\K'(^5g3l	&Bk	D0"4gEnF%Ouxn
6}f
|2/dSe6I$$.A%.Jg'A$wK.lY(pkAGA/b+)
tyYC{"R` @gRRD%?13vXoQ%{dspVlJKS04!_x~"Y*'I|)xt|'PhK\Xi?gVM;Nf=rrR#Oc)SSSh06E\LieJF!
a}v	E=,^`nmbpiy((E[pb6
g4=e.0;qy?Z
	O{8
=)`@:vS^

------------------------------------------------------
샘플링이 끝났습니다.!
-----------------------------------------------------

In [15]:
print ('------------------------------------------------------')
print("최종 샘플링을 시작합니다!")
print ('------------------------------------------------------')
test(2000)
print('\n')
print ('------------------------------------------------------')
print("최종 샘플링이 끝났습니다.!")

------------------------------------------------------
최종 샘플링을 시작합니다!
------------------------------------------------------
berenges old,
Whour oll to he pood
But moust an wils id, wherset trave his, the a caunce muess mesper?

PARTCOER:
Sherees.

DUKEN O:
And bow artou your thou meeded.
I prenon's a so thou asted the hadse.

SARDINA:

BRWOMILO:
So mathou a wardinidengt, The his worred to have sare,
Tos hull brack
Whare
Or Racesmod hance granged, thou hing to whit in ast your whast bow asconcery hum whoay in some.

TRIZO:
Houd what heath to heeing mann pey.

DustO:
In prown to hilent
Fire daich enter, and be the ow loves in got,
Heard and come arlbot.

MLASES:
Whit thou mughter, I thing ospeng, are faill of.
To dore's, with and, deave, res fore corde lords good in I knowch of im lord,
And my a's suis of rown my the hease the as us con,
This thel for hereanceered o letherces: and misttes mughterge cull seak, of stors our't of life your the the be reing his and od sare suret aid, he he 